## Autoencoder simples CONV para deteção de anomalias
Neste caso, para deteção de imagens que não pertencem o MNIST dataset


*Normal Data*


![image_info](https://miro.medium.com/max/1400/1*P7aFcjaMGLwzTvjW3sD-5Q.jpeg)


*Anomaly*


![image_info](https://miro.medium.com/max/1400/1*-bHmKt4JPKIAmmyO47OYnQ.jpeg)


imports

In [ ]:
# pytorch conv for binary classification
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps

import torch
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
    
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor
from torchvision.utils import save_image, make_grid

from torchinfo import summary

In [ ]:
#Constants

#path para guardar o dataset
PATH = './'
PATH_TRAIN = './mnist_train.csv'
PATH_TEST = './mnist_test.csv'

BATCH_SIZE = 128

In [ ]:
#device management 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

device = get_default_device()
print(device)

## 1. Preparar os Dados

In [ ]:
#buscar o dataset utilizando os CSVs e uma classe para o dataset

# definição classe para o dataset
class CSVDataset(Dataset):
    # ler o dataset
    def __init__(self, path_train, path_test):
        # ler o ficheiro csv para um dataframe
        df_train = pd.read_csv(path_train, header=0)
        df_test = pd.read_csv(path_test, header=0)
        # separar os inputs e os outputs
        self.x_train = df_train.values[:, 1:]
        self.x_train = self.x_train.reshape(len(self.x_train), 1, 28, 28)
        xmax, xmin = self.x_train.max(), self.x_train.min()
        self.x_train  = (self.x_train - xmin)/(xmax - xmin)
        self.y_train = df_train.values[:, 0]
        self.x_test = df_test.values[:, 1:]
        self.x_test = self.x_test.reshape(len(self.x_test), 1, 28, 28)
        xmax, xmin = self.x_test.max(), self.x_test.min()
        self.x_test  = (self.x_test - xmin)/(xmax - xmin)
        self.y_test = df_test.values[:, 0]
        # garantir que os inputs e labels sejam floats
        self.x_train = self.x_train.astype('float32')
        self.x_test = self.x_test.astype('float32')
        self.y_train = self.y_train.astype('long')
        self.y_test = self.y_test.astype('long')
        
    # numero de casos de treino no dataset
    def __len_train__(self):
        return len(self.x_train)
     # numero de casos de teste no dataset
    def __len_test__(self):
        return len(self.x_test)
    
    # retornar um caso
    def __getitem_train__(self, idx):
        return [self.x_train[idx], self.y_train[idx]]
     # retornar um caso
    def __getitem_test__(self, idx):
        return [self.x_test[idx], self.y_test[idx]]
    
    # retornar indeces para casos de treino de de teste em formato flat (vetor)
    def get_splits(self):
        x_train  = torch.from_numpy(np.array(self.x_train))
        y_train  = torch.from_numpy(np.array(self.y_train))
        x_test  = torch.from_numpy(np.array(self.x_test))
        y_test  = torch.from_numpy(np.array(self.y_test))
        train = torch.utils.data.TensorDataset(x_train,y_train)
        test = torch.utils.data.TensorDataset(x_test,y_test)
        return train, test 
    
# preparar o dataset
def prepare_data_flat(path_train, path_test):
    # criar uma instancia do dataset
    dataset = CSVDataset(path_train, path_test)
    # calcular split
    train, test = dataset.get_splits()
    # preparar data loaders
    train_dl = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
    test_dl = DataLoader(test, batch_size=BATCH_SIZE, shuffle=True)
    train_dl_all = DataLoader(train, batch_size=len(train), shuffle=False)
    test_dl_all = DataLoader(test, batch_size=len(test), shuffle=False)
    return train_dl, test_dl, train_dl_all, test_dl_all

# preparar os dados
train_dl, test_dl,  train_dl_all, test_dl_all = prepare_data_flat(PATH_TRAIN, PATH_TEST)

## 1.1 Visualizar os Dados

In [ ]:
#Visualização das imagens
def visualize_mnist_images_flat(dl):
    # get one batch of images
    i, (inputs, targets) = next(enumerate(dl))
    print(inputs.shape)
    print(inputs.shape)
    print(inputs.shape)
    # plot some images
    plt.figure(figsize=(8,8))
    for i in range(25):
        # define subplot
        plt.subplot(5, 5, i+1)
        plt.axis('off')
        plt.grid(b=None)
        # plot raw pixel data
        plt.imshow(inputs[i][0], cmap='gray')
    # show the figure
    plt.show()

...

## 3. Ler o modelo previamente treinado em "2_pytorch_AE_CONV_treino_MNIST"

In [ ]:
import models_mnist #modulo python com os modelos       
    
# definir a rede neuronal
model = models_mnist.AE_CONV()

# ler o modelo
SAVED_MODEL = ...
#model= torch.load(SAVED_MODEL)
model= torch.load(SAVED_MODEL, map_location ='cpu')
model.eval()
#visualizar a rede
print(summary(model, input_size=(BATCH_SIZE,  1,28,28), verbose=0))
model.to(device)

## 4. Usar o Autoencoder

In [ ]:
#Podemos utilizar este modelo para deteção de anomalias (imagens que não são digitos)

# Processar a imagem
def process_image(image_path,w,h):
    img = Image.open(image_path)
    width, height = img.size
    # Resize para alteração da dimensão mas a manter o aspect ratio
    img = img.resize((w, int(h*(height/width))) if width < height else (int(w*(width/height)), h))
    # nbter as dimensões novas
    width, height = img.size
    # Definir as coordenadas para o centro de w x h
    left = (width - w)/2
    top = (height - h)/2
    right = (width + w)/2
    bottom = (height + h)/2
    img = img.crop((left, top, right, bottom))
    img = ImageOps.grayscale(img)
    # Converter para array numpy
    img = np.array(img)
    print(f'shape:{img.shape}')
    # Normalizar
    xmax, xmin = img.max(), img.min()
    img  = (img - xmin)/(xmax - xmin)
    # Adicionar uma quarta dimensão ao início para indicar o batch size
    img = img[np.newaxis,:]
    # Converter num tensor torch
    image = torch.from_numpy(img)
    image = image.float()
    #image=image.view(1,w*h) #fazer o flat do 28x28 para ficar como o mnist
    return image

def anomaly_detection(model, img_anomaly, img_list, idx): #img shape (784,1)
    print(img_list.shape)
    print(img_list.dtype) 
    img_list = img_list.to(device)
    img_anomaly= img_anomaly.to(device)   
    pred_img_anomaly,_ = model(img_anomaly)
    print(f'img_anomaly.shape: {img_anomaly.shape}')
    print(f'pred_img_anomaly.shape: {pred_img_anomaly.shape}')
    dist_pred_img = np.linalg.norm(img_anomaly[0].cpu().detach().numpy() - pred_img_anomaly[0].cpu().detach().numpy())  #Distancia de não digito: 22.185663
    print("Distancia de não digito:",dist_pred_img)    
    pred_img_list,_ = model(img_list)
    print(f'pred_img_list.shape: {pred_img_list[idx].shape}')
    dist_img1 = np.linalg.norm(img_list[idx].cpu().detach().numpy() - pred_img_list[idx].cpu().detach().numpy())  #Distancia de não digito: 22.185663
    print("Distancia de  digito1:",dist_img1)  
    
    
ANOMALIA = ...
#ANOMALIA = 'mnist_reconstruction_in.png'

img = process_image(ANOMALIA,28,28)
print(f'img.shape: {img.shape}')
_, (inputs, targets) = next(enumerate(test_dl))
# se a imagem imagem_nao_digito.png não for um digito do genero em que foi treinado então a distancia entre os dois vetores será muito grande.
anomaly_detection(model, img, inputs, 10)